### 自动求导
- 一定要把05的记事本的所有公式都看完，标量对向量求导，向量对标量求导，向量对向量求导的基本公式要看懂
- 实现
- 假设我们想对于函数$y = 2x^Tx$, 关于x的列向量求导

In [2]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])


- 梯度的计算可以通过 **构造计算图**的方法来实现，把计算拆解成一个个操作子，如下图所示,求导的操作可以分为正向求导和反向求导
  - **正向传递**：在链式法则的式子上从后往前算，空间复杂度是O(1)
  - **反向传递**backward propagation：从前往后算，需要存贮中间节点的值，因此空间复杂度是O(n),也是会占用显存的最根本的原因

<img src='images/7_image_1.png' height='200'>

- 在计算y关于x的梯度之前，我们需要一个地方来存储梯度

In [3]:
x.requires_grad_(True)
x.grad

- 计算y

In [4]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

- 调用反向传播函数来自动计算y关于x每个分量的梯度

In [5]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [6]:
x.grad == 4 * x

tensor([True, True, True, True])

- 计算$y = \sum_{i=1}^{n}x_{i}$

In [15]:
# 在默认情况下，pytorch会累计梯度， 我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

- 在深度学习中，一般使用的都是标量对向量的求导，因为深度学习任务是要预测一个类似数值一样的标量一样的东西，所以基本上都是用结果的标量来对参数向量或者矩阵进行求导(个人见解)

- 将某些计算移动到计算图之外

- detach是tensor中一个很有用的属性，用于从计算图中分离张量，detach调用后会创建一个新的张量，该张量与原始张量共享相同的数据，但不再参与梯度计算

In [19]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x
z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [20]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

- 即使构建函数的计算图需要通过python控制流（例如，条件，循环或者函数调用），我们依然可以计算得到变量的梯度

In [22]:
def f(a:torch.Tensor):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

a.grad == d / a
    

tensor(True)